In [240]:
# !apt-get update
# !apt-get install wkhtmltopdf pandoc libfontconfig1 libxrender1 libxext6
# !apt --fix-broken install

In [1]:
!pip install OpenAI markdown mdx_truly_sane_lists weasyprint python-dotenv

In [2]:
import os

from dotenv import load_dotenv
load_dotenv()  # Load the environment variables from .env file

# for Google Colab
#from google.colab import userdata
#os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

#print(os.environ['OPENAI_API_KEY'])

True

# Setup

In [243]:
from openai import OpenAI

client = OpenAI()

# retrieve the Assistant
my_assistant = client.beta.assistants.retrieve("asst_3WQiw8VJ91XWikievImBqwrZ")

# create a thread
my_thread = client.beta.threads.create()


In [244]:
import time

def create_run_and_display_response(client, my_thread, my_assistant, content, file_ids=None):
    """
    Create a message, run it, and display the response.

    Args:
    client (Client): The API client.
    my_thread (Thread): The thread to post the message to.
    my_assistant (Assistant): The assistant to use for the run.
    content (str): The content of the message.
    file_ids (list, optional): List of file IDs to attach to the message.

    Returns:
    str: The response content.

    Raises:
    ValueError: If any of the parameters are invalid.
    RuntimeError: If the run does not complete successfully.
    """

    if not all([client, my_thread, my_assistant, content]):
        raise ValueError("Invalid parameters provided.")

    extra_instructions = """ ONLY return the content as markdown (with no delimiters) and with no commentary what so ever."""

    # Create the message
    thread_message = client.beta.threads.messages.create(
        my_thread.id,
        role="user",
        content=content + extra_instructions,
        file_ids=file_ids or []
    )

    # Hard code the values for max_retries and wait_seconds
    max_retries = 120
    wait_seconds = 0.5

    # Create a run
    run = client.beta.threads.runs.create(
        thread_id=my_thread.id,
        assistant_id=my_assistant.id
    )

    # Check the run's status
    for _ in range(max_retries):
        run = client.beta.threads.runs.retrieve(
            thread_id=my_thread.id,
            run_id=run.id)
        if run.status == "completed":
            break
        time.sleep(wait_seconds)
    else:
        raise RuntimeError("Run did not complete successfully.")

    # Retrieve and return the response
    all_messages = client.beta.threads.messages.list(thread_id=my_thread.id)
    return all_messages.data[0].content[0].text.value + "\n\n"



# Initial File Upload and Prompt

In [245]:
# get the job description from GitHub
import time


file_path = 'jobdescription.txt'
# infra engineer example
#!wget -O file_path https://raw.githubusercontent.com/androb/deliverables/main/interviewplaybook/examples/infra-engineer/job-description.md
# medical assistant example
!wget -O 'jobdescription.txt' https://raw.githubusercontent.com/androb/deliverables/main/interviewplaybook/examples/medical-assistant/job-description.md

# load it into a variable for later use
with open(file_path, 'r') as file:
    job_description = file.read()

# infra engineer example
# !wget -O companyvalues.txt https://raw.githubusercontent.com/androb/deliverables/main/interviewplaybook/examples/infra-engineer/company-values.md
# medical assistant example
!wget -O companyvalues.txt https://github.com/androb/deliverables/blob/main/interviewplaybook/examples/medical-assistant/about-company.md
# time.sleep(5)


# medical assistant example
# !wget -O jobdescription.txt https://raw.githubusercontent.com/androb/deliverables/main/interviewplaybook/examples/medical-assistant/job-description.md

# Upload the file
file_company_values = client.files.create(
  file=open("companyvalues.txt", "rb"),
  purpose="assistants"
)

# Upload the file
file_job_description = client.files.create(
  file=open("jobdescription.txt", "rb"),
  purpose="assistants"
)
time.sleep(5)



--2023-12-19 20:16:51--  https://raw.githubusercontent.com/androb/deliverables/main/interviewplaybook/examples/medical-assistant/job-description.md
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2483 (2.4K) [text/plain]
Saving to: ‘jobdescription.txt’

jobdescription.txt  100%[===================>]   2.42K  --.-KB/s    in 0s      

2023-12-19 20:16:51 (30.1 MB/s) - ‘jobdescription.txt’ saved [2483/2483]

--2023-12-19 20:16:51--  https://github.com/androb/deliverables/blob/main/interviewplaybook/examples/medical-assistant/about-company.md
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16670 (16K) [text/plain]
Saving to: ‘company

# Priming the Pump

In [246]:
# Refresh the context to ensure it has read the files
pump_primer = create_run_and_display_response(client,my_thread,my_assistant,"""
            Write a detailed description of the company values file, using the company name.
            Then, write the job title contained in the job description file.
            Then, create the list of technical competencies, which MUST cover the areas mentioned in the
            requirements in the job description (and no more). Create the list of behavioral
            competencies, with a focus on the company values but still covering all requirements in the
            job description. Where possible, describe a competency in a way that aligns with the company values.
            Each competency should have a title followed by a short description.
            """, [file_job_description.id, file_company_values.id])
print(pump_primer)


Values:

- Innovation - Embracing change and continuous improvement, striving to raise quality of care and shape the future of medicine.
- Connection - Cultivating community and care to nurture trust, respect, belonging, and shared purpose.
- Teamwork - Collaborating across all levels and departments to unlock greater impact and fulfillment.
- Mentorship - Fostering professional growth for impactful and fulfilling careers.
- Participation - Valuing diverse backgrounds and contributions to improve decision-making and outcomes.
- Action - Uniting diverse perspectives and expertise for the greater good【11†source】.

Job title: Medical Assistant II【19†source】.

Technical Competencies:

1. Clinical Procedures
   - Mastery in a range of clinical tasks and procedures to support patient care delivery, demonstrating precision and adherence to best practices.

2. Patient Communication Skills
   - Excellent skills in communicating with patients, including the ability to explain medical procedures 

# Create Introduction

In [247]:
introduction = create_run_and_display_response(client,my_thread,my_assistant,"Generate the Introduction with the Candidate Profle (only if applicable) and Interview Structure. Be sure to include mention of the competencies in the interview structure.")
print(introduction)

# Interview Playbook for Medical Assistant II

## Introduction

This guide is designed to steer the interview process for the role of Medical Assistant II. As an essential part of the healthcare team at Stanford Health Care - University Healthcare Alliance, this role is integral to maintaining the standards of excellence and innovation our organization is known for. The objective is to use structured, values-driven interview techniques to identify candidates who demonstrate technical proficiency, align with our core values—Innovation, Connection, Teamwork, Mentorship, Participation, and Action—and exhibit the potential to contribute to our dynamic, high-performance team.

### Interview Structure

The interview should range from 45 to 60 minutes and be structured as follows:

1. **Introduction (5 minutes)** - A brief welcome and overview of the company, the role, and the interview plan.
   
2. **Technical Assessment (20 minutes)** - Evaluate technical competencies: Clinical Procedures, 

# Create Evaluation Criteria

In [274]:
eval_criteria = create_run_and_display_response(client,my_thread,my_assistant,"""
    Generate the Evaluation Critera section.
    """)
print(eval_criteria)

## Evaluation Criteria

The following job scorecard is designed to evaluate the candidate's competency across technical skills, behavioral characteristics, and their overall fit within the team.

| Competency                     | Assessment                | Comments                                 |
|--------------------------------|---------------------------|------------------------------------------|
| Clinical Procedures            | [ ] 1 [ ] 2 [ ] 3 [ ] 4 [ ] 5 |                                          |
| Patient Communication Skills   | [ ] 1 [ ] 2 [ ] 3 [ ] 4 [ ] 5 |                                          |
| Computer Literacy              | [ ] 1 [ ] 2 [ ] 3 [ ] 4 [ ] 5 |                                          |
| Basic Life Support (BLS)       | [ ] 1 [ ] 2 [ ] 3 [ ] 4 [ ] 5 |                                          |
| Multi-Tasking Ability          | [ ] 1 [ ] 2 [ ] 3 [ ] 4 [ ] 5 |                                          |
| Service Orientation            | [ ] 1 [

# Create Technical Assessment

In [249]:
technical_assessment_1 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the first Technical Assessment")
print(technical_assessment_1)


## Technical Assessment

Delving into the candidate's technical skills is vital to confirm they can meet the demands of the Medical Assistant II role, particularly their expertise in clinical procedures.

### Technical Competence 1: Clinical Procedures

_Clinical procedures are at the heart of the Medical Assistant II role. Candidates must possess comprehensive skills in performing a variety of medical tasks, including taking vital signs, preparing patients for exams, and assisting with procedures. Proficiency here directly impacts patient care quality._

#### Lead Question

"Can you walk me through a time when you had to perform a complex clinical procedure under significant time constraints? Specifically, how did you ensure the quality of care was maintained?"

#### Probing Questions

- "What strategies do you use to stay current with the latest clinical practices and techniques?"
- "Describe a situation where you had to adapt a standard procedure to better suit a patient's needs. Wh

In [250]:
technical_assessment_2 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 2nd Technical Assessment. Don't include the main section header.")
print(technical_assessment_2)

### Technical Competence 2: Patient Communication Skills

_Effective communication with patients is a cornerstone of healthcare. Medical Assistants must be adept at explaining medical terms in an understandable way, showing empathy, and gathering crucial patient information._

#### Lead Question

"Describe a time when you faced a communication challenge with a patient who was anxious or upset. How did you manage to convey important information and address their concerns?"

#### Probing Questions

- "How do you ensure that you have understood a patient's concerns and that they have understood your instructions?"
- "What techniques do you use to break down complex medical jargon for patients of different backgrounds and education levels?"
- "Can you tell me about a time when your communication skills directly contributed to a patient's treatment or experience?"
- "How do you handle situations where a patient is non-compliant with instructions or resistant to communication?"

#### Scenari

In [251]:
technical_assessment_3 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 3rd Technical Assessment. Don't include the main section header.")
print(technical_assessment_3)

### Technical Competence 3: Computer Literacy

_In today's healthcare environment, proficiency in electronic medical records (EMR) and general computer applications is critical for managing patient information efficiently and ensuring accuracy._

#### Lead Question

"Can you share an experience where your computer skills allowed you to improve a process or enhance patient care within the healthcare setting?"

#### Probing Questions

- "What software systems are you familiar with, and how have you utilized them in a clinical setting?"
- "Describe a time when you encountered a technical issue with an EMR system. How did you resolve it?"
- "How do you ensure data accuracy and security when managing patient records?"
- "Can you discuss a particular instance where you had to learn a new healthcare software application quickly? What was your approach?"

#### Scenario

"You are at the clinic and the EMR system has just been updated to include a new feature for patient follow-up tracking. The 

In [252]:
technical_assessment_4 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 4th Technical Assessment. Don't include the main section header.")
print(technical_assessment_4)

### Technical Competence 4: Basic Life Support (BLS)

_Basic Life Support certification is essential, as it ensures that a Medical Assistant can provide immediate care in life-threatening situations and support patient safety._

#### Lead Question

"Tell me about a time when you had to utilize your Basic Life Support training in a real-life situation. What steps did you take and what was the outcome?"

#### Probing Questions

- "How do you maintain your BLS skills to ensure you are always prepared for an emergency?"
- "Describe how you would handle a situation where a patient begins to choke or shows signs of cardiac arrest."
- "What is your process for staying up-to-date with the latest BLS protocols?"
- "Can you provide an example where your quick thinking and BLS skills significantly impacted a patient's survival chance?"

#### Scenario

"Imagine you are in the clinic, and a patient waiting for their appointment suddenly collapses and is unresponsive. There's no pulse, and the patie

In [253]:
technical_assessment_5 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 5th Technical Assessment. Don't include the main section header.")
print(technical_assessment_5)

### Technical Competence 5: Multi-Tasking Ability

_The ability to effectively juggle multiple tasks is critical for a Medical Assistant II, ensuring a smooth and efficient workflow within the healthcare environment._

#### Lead Question

"Describe a particularly busy day at work where you had to manage several tasks at once. How did you prioritize, and what was the result of your multi-tasking?"

#### Probing Questions

- "How do you organize your daily tasks to maximize efficiency without sacrificing quality of care?"
- "What tools or methods do you employ to keep track of multiple patients and their needs simultaneously?"
- "Can you give an example of a time when you had to adjust your priorities on the fly due to an unexpected situation? How did you handle it?"
- "What strategies do you use to maintain accuracy and attention to detail in a fast-paced work environment?"

#### Scenario

"You are the only Medical Assistant on duty, and the clinic is unexpectedly short-staffed. You hav

In [254]:
technical_assessment = technical_assessment_1 + technical_assessment_2 + technical_assessment_3 + technical_assessment_4 + technical_assessment_5

# Create Behavioral Assessment

In [255]:
behavioral_assessment_1 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 1st Behavioral Assessment section.")
print(behavioral_assessment_1)

## Behavioral Assessment

Behavioral competencies are essential to determine how a candidate's values, personality, and work style align with the company culture and values.

### Behavioral Competence 1: Service Orientation (Innovation & Action)

_Service orientation in the context of innovation and action is about a proactive approach to patient care, always seeking to enhance the patient experience and contribute to healthcare advancements._

#### Lead Question

"Describe an instance where you went above and beyond the standard protocols to improve a patient's experience. How did your actions reflect the company's values of innovation and taking action?"

#### Probing Questions

- "What strategies do you employ to consistently provide high-quality patient service?"
- "Can you give an example of a time when you contributed an idea that improved the service in your department?"
- "How do you balance patient needs with the efficiency and continuous improvement of services?"
- "When have

In [256]:
behavioral_assessment_2 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 2nd Behavioral Assessment section. Don't include the main section header.")
print(behavioral_assessment_2)

### Behavioral Competence 2: Team Cooperation (Teamwork & Participation)

_Team cooperation is about working harmoniously with colleagues across different functions, promoting teamwork and valuing diverse views and participation to achieve common goals._

#### Lead Question

"Can you share a situation where your ability to work effectively with a team led to a successful patient outcome or project completion?"

#### Probing Questions

- "How do you handle disagreements within a team, and what approach do you take to find common ground?"
- "Describe a project that required input from various departments. How did you facilitate collaboration and contribution?"
- "In what ways do you encourage and foster participation from all team members, especially those who are typically less vocal?"
- "Tell me about a time when the collective effort of a team you were a part of led to an innovation or significant improvement."

#### Scenario

"Your team is facing a challenging situation where multipl

In [257]:
behavioral_assessment_3 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 3rd Behavioral Assessment section. Don't include the main section header.")
print(behavioral_assessment_3)

### Behavioral Competence 3: Communication Competence (Connection & Mentorship)

_Communication competence involves effectively exchanging information, supporting a culture of connection, and enabling mentorship by being approachable and clear._

#### Lead Question

"Tell me about a time when effective communication on your part helped to build a stronger connection within your team or with a patient."

#### Probing Questions

- "How do you ensure that your communication is inclusive and accessible to everyone, regardless of their background?"
- "What methods have you used to foster a mentoring relationship with someone less experienced in your field?"
- "Describe a situation where you had to deliver difficult news. How did you handle it?"
- "Can you share an experience where active listening improved a work-related outcome?"

#### Scenario

"You are working with a new hire who is struggling to grasp the nuances of the patient management system. Formulate a communication strategy that 

In [258]:
behavioral_assessment_4 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 4th Behavioral Assessment section. Don't include the main section header.")
print(behavioral_assessment_4)

### Behavioral Competence 4: Ethical and Inclusive Conduct (Participation & Action)

_Ethical and inclusive conduct is fundamental to creating an environment of trust and respect. It involves commitment to ethical standards and fostering inclusivity by valuing diverse participation and taking action._

#### Lead Question

"Explain a time when you had to make a decision in a work situation that tested your ethical boundaries. How did you approach the issue, and what was the outcome?"

#### Probing Questions

- "How do you ensure that all patients are treated with equal care and attention, regardless of their background?"
- "Describe an instance when you advocated for a colleague or patient who was being treated unfairly. What actions did you take?"
- "What do you believe is your role in promoting diversity and inclusion within the healthcare environment?"
- "Can you provide an example of how you actively contribute to a workplace culture that supports participation and action?"

#### Sc

In [259]:
behavioral_assessment_5 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 5th Behavioral Assessment section. Don't include the main section header.")
print(behavioral_assessment_5)

### Behavioral Competence 5: Professional Development (Mentorship & Innovation)

_Professional development refers to the continuous pursuit of knowledge and skills essential for career advancement. This competency is intertwined with the values of mentorship and innovation, promoting a culture of learning and growth._

#### Lead Question

"Discuss a time when you took the initiative to enhance your professional skills. How did this development translate into better performance or innovation in your work?"

#### Probing Questions

- "How do you stay updated with the latest trends and practices in healthcare, and how do you apply this knowledge to your role?"
- "Can you describe a scenario where you mentored someone to help them grow professionally?"
- "What strategies do you use to encourage a culture of professional development among your peers?"
- "Share an innovative idea or project you initiated that had a positive impact on your workplace."

#### Scenario

"You've been offered the 

In [260]:
behavioral_assessment = behavioral_assessment_1 + behavioral_assessment_2 + behavioral_assessment_3 + behavioral_assessment_4 + behavioral_assessment_5

# Create Practical Assignment

In [261]:
practical_assignment = create_run_and_display_response(client,my_thread,my_assistant,"Generate the Practical Assignment section.")
print(practical_assignment)


## Practical Assignment

The practical assignment for the Medical Assistant II position is designed to test the candidate's hands-on skills and their ability to apply technical knowledge in a real-world scenario.

### Description

The candidate will be provided with a simulated patient case that includes a set of symptoms and medical history. They will be expected to perform relevant clinical procedures, demonstrate proper patient communication, and use the electronic medical record system to document their actions.

### Objectives

- To assess the candidate's competency in executing clinical tasks.
- To evaluate their ability to communicate effectively with patients.
- To confirm their proficiency in using healthcare-related software.

### Evaluation Criteria

Candidates will be evaluated on the following:

- Accuracy and efficiency in completing clinical tasks.
- Effectiveness in patient communication, ensuring clarity and empathy.
- Mastery of navigating and entering data into the E

# Create Team Interaction

In [262]:
team_interaction = create_run_and_display_response(client,my_thread,my_assistant,"Generate the Team Interaction section.")
print(team_interaction)

## Team Fit

The purpose of assessing team interaction is to determine how well the candidate integrates with the existing team and contributes to a positive and productive work environment.

### Team Interaction Rationale

Candidates will interact with various members of the healthcare team to demonstrate their ability to collaborate, communicate, and work effectively within a group setting. These interactions will also provide insights into how the candidate handles teamwork and shared responsibilities.

### Discussion Points

- How the candidate engages in teamwork and collaborative problem solving.
- Observation of the candidate's communication skills within a team dynamic.
- Assessment of the candidate’s ability to adapt to different team members’ working styles.
- Evaluation of how the candidate handles constructive feedback from team members.

### Activities

- Participate in a brief team meeting to discuss patient care strategies.
- Collaborate on a mini-project or task relevan

# Create Candidate Q&A

In [263]:
candidate_questions = create_run_and_display_response(client,my_thread,my_assistant,"""
    Generate the Candidate Q&A section. Include example answers.
    """)
print(candidate_questions)

## Candidate Q&A

Candidates often have questions about the role, team, and company during the interview process. It's important to provide comprehensive, honest answers to help them assess whether the opportunity aligns with their professional goals and values.

### Questions and Answers

1. **What are the opportunities for professional development within the company?**
   - "We are committed to continuous learning and growth. You'll have access to various training programs, workshops, and mentorship opportunities to develop your skills further and advance your career."

2. **Can you tell me more about the team I'll be working with?**
   - "You'll be part of a diverse and dedicated team that's collaborative and supportive. Our team members come from a range of backgrounds, and we all share a commitment to providing excellent patient care."

3. **What type of patient population does the clinic serve?**
   - "Our clinic serves a wide array of patient demographics, offering a unique oppo

# Finalize the Playbook HTML

## Create a markdown version

For use in GitHub as an example

In [275]:
playbook_md = (introduction + "\n\n" +
            job_description + "\n\n" +
            eval_criteria + "\n\n" +
            technical_assessment + "\n\n" +
            behavioral_assessment + "\n\n" +
            practical_assignment + "\n\n" +
            candidate_questions + "\n\n" +
            team_interaction + "\n\n" )

# Write playbook to a text file
with open('playbook.md', 'w') as file:
    file.write(playbook_md)

## Convert the markdown to HTML

We are doing this in chunks so that we can do more useful things with it in the HTML output

In [276]:
import markdown

# Convert Markdown to HTML
html_playbook = markdown.markdown(playbook_md, extensions=['extra','mdx_truly_sane_lists'])

# Convert Markdown to HTML
html_job_description = markdown.markdown(job_description, extensions=['extra','mdx_truly_sane_lists'])
html_introduction = markdown.markdown(introduction, extensions=['extra','mdx_truly_sane_lists'])
html_technical_assessment = markdown.markdown(technical_assessment, extensions=['extra','mdx_truly_sane_lists'])
html_behavioral_assessment = markdown.markdown(behavioral_assessment, extensions=['extra','mdx_truly_sane_lists'])
html_practical_assignment = markdown.markdown(practical_assignment, extensions=['extra','mdx_truly_sane_lists'])
html_candidate_questions = markdown.markdown(candidate_questions, extensions=['extra','mdx_truly_sane_lists'])
html_team_interaction = markdown.markdown(team_interaction, extensions=['extra','mdx_truly_sane_lists'])
html_eval_criteria = markdown.markdown(eval_criteria, extensions=['extra','mdx_truly_sane_lists'])


html_technical_assessment_1 = markdown.markdown(technical_assessment_1, extensions=['extra','mdx_truly_sane_lists'])
html_technical_assessment_2 = markdown.markdown(technical_assessment_2, extensions=['extra','mdx_truly_sane_lists'])
html_technical_assessment_3 = markdown.markdown(technical_assessment_3, extensions=['extra','mdx_truly_sane_lists'])
html_technical_assessment_4 = markdown.markdown(technical_assessment_4, extensions=['extra','mdx_truly_sane_lists'])
html_technical_assessment_5 = markdown.markdown(technical_assessment_5, extensions=['extra','mdx_truly_sane_lists'])

html_behavioral_assessment_1 = markdown.markdown(behavioral_assessment_1, extensions=['extra','mdx_truly_sane_lists'])
html_behavioral_assessment_2 = markdown.markdown(behavioral_assessment_2, extensions=['extra','mdx_truly_sane_lists'])
html_behavioral_assessment_3 = markdown.markdown(behavioral_assessment_3, extensions=['extra','mdx_truly_sane_lists'])
html_behavioral_assessment_4 = markdown.markdown(behavioral_assessment_4, extensions=['extra','mdx_truly_sane_lists'])
html_behavioral_assessment_5 = markdown.markdown(behavioral_assessment_5, extensions=['extra','mdx_truly_sane_lists'])



 ## Create the style for the document

Borrowed colors from this [color palette](https://color.adobe.com/PALETTE-mini-06:-Transparent-color-theme-15d8c4a6-b06a-4530-bf5b-3fc8102ccd69)

In [277]:
styles= """

   body {
        font-family: Source Sans Pro;
        font-size: 11pt;
        font-weight: 200;
        font-stretch: 75% 125%;
        color:#0D0D0D;
    }
    @page {
        margin: 0.65in;
        size: Letter;
        @bottom-right {
          content: counter(page);
          font-size: 11pt;
          color: #73706C;
        }
    }
    @page :first {
        @bottom-right {
          content: none;
        }
    }


    h1, h2, h3, h4 {
        font-family: Libre Baskerville;
        margin-top: 1em;
        margin-bottom: 0.5em;
    }
    h1 {
        font-size: 24pt;
        margin-top: 3em;
        page-break-after: avoid;
    }
    .intro h1{
        font-size: 50pt;
        margin-top: 3em;
        page-break-after: always;
    }
    @media print {
      /* Define styles for the cover page */
      .title {
        position: fixed;
        left: 0;
        top: 0;
        width: 100%;
        height: 100%;
        background-color: #F2F2F2; /* Change as needed */
        color: #0D0D0D; /* Change as needed */
        text-align: center;
        padding: 50px;
      }
    }
    .publication-date {
      position: absolute;
      bottom: 10px; /* adjust as needed */
      width: 100%;
      text-align: left;
      font-size: 11pt; /* adjust as needed */
      color: #73706C; /* adjust as needed */
    }


    h2 {
        font-size: 18pt;
        page-break-after: avoid;
    }
    h3 {
        font-size: 14pt;
        page-break-after: avoid;
    }
    h4 {
        font-size: 12pt;
        page-break-after: avoid;
    }
    p {
        margin-bottom: 1em;
    }


    h1, h2, h3, h4 {
        page-break-after: avoid;
    }

    p, table, figure {
        page-break-inside: avoid;
    }
    table {
        width: 100%;
        border-collapse: collapse;
        margin-bottom: 1em;
        page-break-before: avoid;
    }
    th, td {
        border: 1px solid #ddd;
        padding: 8px;
        text-align: left;
        font-size: 10pt;
    }
    th {
        background-color: #f2f2f2;
        font-weight: bold;
    }
    tr:nth-child(even) {{background-color: #f9f9f9;}}

    ul, ol {
        margin-bottom: 1em;
        margin-left: 0;
        padding-left:1.5em;
    }
    li {
        margin-bottom: 0.5em;
        margin-left: 0em;
    }

    .main {
      page-break-before: always;
    }

    .dotted-line {
        border-bottom: 1px dotted black;
        width: 100%;
        margin-top: 20px; /* This ensures spacing between each line */
        margin-bottom: 20px; /* Adjust if needed */
    }

    @media print {
        .dotted-line {
            margin-top: 20px; /* Adjust as needed for printing */
            margin-bottom: 20px; /* Adjust as needed for printing */
        }
    }

    .rating-container {
        display: flex;
        align-items: flex-end;
        margin-top: 20px;
        margin-bottom: 20px;
    }

    .rating-container label {
        margin-right: 10px;
        font-size: 10pt;
        font-family: Source Sans Pro;
    }

    .short-dotted-line {
        border-bottom: 1px dotted black;
        width: 150px; /* Adjust the width as needed */
        margin-left: 5px;
    }}

    @media print {
        .short-dotted-line {
            width: 150px; /* Adjust for printing if needed */
        }
    }
"""

## Create some useful HTML snippets

In [278]:

html_notes = """<h4>Notes</h4><div class='notes'>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="rating-container">
    <label for="rating-line">Rating (1-5):</label>
    <div id="rating-line" class="short-dotted-line"></div>
    </div>
</div>"""

html_short_notes = """<h4>Notes</h4><div class='notes'>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="rating-container">
    <label for="rating-line">Rating (1-5):</label>
    <div class="short-dotted-line"></div>
    </div>
</div>"""

from datetime import datetime

# Getting today's date in a nicely formatted string for US English
publication_date = datetime.now().strftime("%B %d, %Y")

## Build the HTML

In [279]:
# Define the HTML template
html_content = rf"""
<meta charset="utf-8">
<html>
<head>
  <link href="https://fonts.googleapis.com/css2?family=Source%20Sans%20Pro:wght@400;700" rel="stylesheet">
  <link href="https://fonts.googleapis.com/css2?family=Libre%20Baskerville:wght@400;700" rel="stylesheet">
  <style>
  {styles}
  </style>
</head>
<body>
    <section class='intro'>
    <div class="publication-date">Published on {publication_date} by the <a href="https://github.com/androb/deliverables/tree/main/interviewplaybook">Interview Playbook Generator</a></div>
    {html_introduction}
    </section>
    <section class='main'>
    {html_job_description}
    </section>
    <section class='main'>
    {html_eval_criteria}
    </section>
    <section class='main'>
      {html_technical_assessment_1}
     {html_short_notes}
    </section>
    <section class='main'>
      {html_technical_assessment_2}
     {html_notes}
    </section>
        <section class='main'>
      {html_technical_assessment_3}
     {html_notes}
    </section>
        <section class='main'>
      {html_technical_assessment_4}
     {html_notes}
    </section>
        <section class='main'>
      {html_technical_assessment_5}
     {html_notes}
    </section>
    <section class='main'>
    {html_behavioral_assessment_1}
     {html_short_notes}
    </section>
    <section class='main'>
    {html_behavioral_assessment_2}
     {html_notes}
    </section>
        <section class='main'>
    {html_behavioral_assessment_3}
     {html_notes}
    </section>
        <section class='main'>
    {html_behavioral_assessment_4}
     {html_notes}
    </section>
        <section class='main'>
    {html_behavioral_assessment_5}
     {html_notes}
    </section>
    <section class='main'>
    {html_practical_assignment}
    {html_notes}
    </section>
    <section class='main'>
    {html_team_interaction}
    {html_notes}
    </section>
    <section class='main'>
    {html_candidate_questions}
    </section>

</body>
</html>"""

# Write HTML string to a file
with open('playbook.html', 'w') as file:
    file.write(html_content)



Display the HTML content here so we can review it

In [280]:
# Display the Markdown-formatted text in the notebook
from IPython.display import display, HTML
display(HTML(html_content))

Competency,Assessment,Comments
Clinical Procedures,[ ] 1 [ ] 2 [ ] 3 [ ] 4 [ ] 5,
Patient Communication Skills,[ ] 1 [ ] 2 [ ] 3 [ ] 4 [ ] 5,
Computer Literacy,[ ] 1 [ ] 2 [ ] 3 [ ] 4 [ ] 5,
Basic Life Support (BLS),[ ] 1 [ ] 2 [ ] 3 [ ] 4 [ ] 5,
Multi-Tasking Ability,[ ] 1 [ ] 2 [ ] 3 [ ] 4 [ ] 5,
Service Orientation,[ ] 1 [ ] 2 [ ] 3 [ ] 4 [ ] 5,
Team Cooperation,[ ] 1 [ ] 2 [ ] 3 [ ] 4 [ ] 5,
Communication Competence,[ ] 1 [ ] 2 [ ] 3 [ ] 4 [ ] 5,
Ethical and Inclusive Conduct,[ ] 1 [ ] 2 [ ] 3 [ ] 4 [ ] 5,
Professional Development,[ ] 1 [ ] 2 [ ] 3 [ ] 4 [ ] 5,


# Create the Documents

Create the Word document

In [281]:
# Write HTML string to a file
with open('playbook.html', 'w') as file:
    file.write(html_content)

# Convert HTML to Docx
!pandoc -s playbook.html -o playbook.docx

# from google.colab import files
# files.download('playbook.docx')

Create the PDF

In [282]:
from weasyprint import HTML

# Convert HTML to PDF using WeasyPrint
#HTML('playbook.html').write_pdf('playbook.pdf', stylesheets=['style.css'], presentational_hints=True)
HTML('playbook.html').write_pdf('playbook.pdf')

from google.colab import files
files.download('playbook.pdf')

# Convert HTML to alternative PDF
#!pandoc -s playbook.html --pdf-engine=wkhtmltopdf -o playbook.pdf

# Convert HTML to alternative PDF
#!wkhtmltopdf --page-size Letter --footer-html footer.html --margin-top 15mm --margin-bottom 15mm --margin-left 15mm --margin-right 15mm playbook.html playbook.pdf

# Convert HTML to PDF
##import pdfkit
# pdfkit.from_string(html_content, 'playbook.pdf')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>